In [ ]:
# !pip install plotly

import plotly.graph_objects as go
import numpy as np

def plot_map(lon, lat, spin=0, tilt=0, roll=0):
    fig = go.Figure(go.Scattergeo())
    fig.update_geos(projection_type="natural earth")
    fig.update_geos(
        center=dict(lon=lon, lat=lat),
        projection_rotation=dict(lon=spin, lat=tilt, roll=roll)
    )
    fig.update_geos(
        visible=True, resolution=110,
        showcountries=True, countrycolor="RebeccaPurple",
        showland=True, landcolor="LightGreen",
        showocean=True, oceancolor="LightBlue",
    )
    fig.update_geos(lataxis_showgrid=True, lonaxis_showgrid=True)
    return fig


def antipode(lon, lat):
    return (lon+180)%360, -lat

antipode(0,90)  # North Pole

(180, -90)

In [184]:
fig0 = plot_map(0,0,0)
fig0.add_trace(go.Scattergeo(
    lon=[50],
    lat=[40],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Annotation Point'
))
fig0.add_trace(go.Scattergeo(
    lon=[230],
    lat=[50],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Annotation Point'
))
fig0.add_trace(go.Scattergeo(
    lon=[50],
    lat=[-50],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Annotation Point'
))
fig0.show()

In [34]:
fig1 = plot_map(50, 40, spin=50, tilt=40, roll=40)
fig1.show()

In [111]:
fig2 = plot_map(50, 40, spin=50, tilt=40, roll=0)
fig2.add_trace(go.Scattergeo(  # North Pole (0, 90) -> (-130, 50)
    lon=[-130],
    lat=[50],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='North'
))
fig2.add_trace(go.Scattergeo(  # South Pole (0, -90) -> (50, -50)
    lon=[50],
    lat=[-50],
    mode='markers',
    marker=dict(size=10, color='blue'),
    name='South'
))
fig2.show()

In [185]:
def transform_point(lon, lat, center_lon=50, center_lat=40, roll=40):
    """
    Apply the same rotation transformation to a point that's applied to the map
    """
    # Convert to radians
    lon_rad = np.radians(lon - center_lon)
    lat_rad = np.radians(lat)
    center_lat_rad = np.radians(center_lat)
    roll_rad = np.radians(roll)
    
    # Apply rotation around the center
    # This is a simplified transformation - for precise results you'd need
    # full spherical coordinate transformation
    
    # Rotate around vertical axis (longitude rotation)
    new_lon = lon_rad
    
    # Rotate around horizontal axis (latitude rotation)  
    new_lat = lat_rad - center_lat_rad
    
    # Apply roll rotation (around line of sight)
    if roll != 0:
        cos_roll = np.cos(roll_rad)
        sin_roll = np.sin(roll_rad)
        temp_lon = new_lon * cos_roll - new_lat * sin_roll
        temp_lat = new_lon * sin_roll + new_lat * cos_roll
        new_lon, new_lat = temp_lon, temp_lat
    
    # Convert back to degrees
    return float(np.degrees(new_lon) + center_lon), float(np.degrees(new_lat) + center_lat)


def inverse_transform_point(visual_lon, visual_lat, center_lon=50, center_lat=40, roll=40):
    """
    Transform from visual map coordinates back to geographic coordinates
    
    visual_lon, visual_lat: where you want the point to appear on your rotated map
    center_lon, center_lat, roll: your map's rotation parameters
    
    Returns: the actual geographic coordinates to plot
    """
    # Convert to radians
    v_lon_rad = np.radians(visual_lon)
    v_lat_rad = np.radians(visual_lat)
    c_lon_rad = np.radians(center_lon) 
    c_lat_rad = np.radians(center_lat)
    roll_rad = np.radians(roll)
    
    # Apply inverse roll rotation first
    if roll != 0:
        cos_r = np.cos(-roll_rad)  # negative for inverse
        sin_r = np.sin(-roll_rad)
        
        temp_lon = v_lon_rad * cos_r - v_lat_rad * sin_r
        temp_lat = v_lon_rad * sin_r + v_lat_rad * cos_r
        v_lon_rad, v_lat_rad = temp_lon, temp_lat
    
    # Apply inverse center translation
    actual_lon = np.degrees(v_lon_rad) + center_lon
    actual_lat = np.degrees(v_lat_rad) + center_lat
    
    return float(actual_lon), float(actual_lat)


import numpy as np

def inverse_transform_point_spherical(visual_lon, visual_lat, center_lon, center_lat, roll=0):
    """
    More accurate inverse transformation using spherical coordinates
    """
    # Convert to radians
    v_lon_rad = np.radians(visual_lon)
    v_lat_rad = np.radians(visual_lat)
    c_lon_rad = np.radians(center_lon)
    c_lat_rad = np.radians(center_lat) 
    roll_rad = np.radians(roll)
    
    # Convert to Cartesian coordinates on unit sphere
    x = np.cos(v_lat_rad) * np.cos(v_lon_rad)
    y = np.cos(v_lat_rad) * np.sin(v_lon_rad)
    z = np.sin(v_lat_rad)
    
    # Apply inverse roll rotation around z-axis (viewing direction)
    if roll != 0:
        cos_r = np.cos(-roll_rad)
        sin_r = np.sin(-roll_rad)
        x_new = x * cos_r - y * sin_r
        y_new = x * sin_r + y * cos_r
        x, y = x_new, y_new
    
    # Apply inverse latitude rotation (around y-axis)
    if center_lat != 0:
        cos_lat = np.cos(-c_lat_rad)
        sin_lat = np.sin(-c_lat_rad)
        x_new = x * cos_lat - z * sin_lat
        z_new = x * sin_lat + z * cos_lat
        x, z = x_new, z_new
    
    # Apply inverse longitude rotation (around z-axis)
    if center_lon != 0:
        cos_lon = np.cos(-c_lon_rad)
        sin_lon = np.sin(-c_lon_rad)
        x_new = x * cos_lon - y * sin_lon
        y_new = x * sin_lon + y * cos_lon
        x, y = x_new, y_new
    
    # Convert back to spherical coordinates
    actual_lat = np.degrees(np.arcsin(np.clip(z, -1, 1)))
    actual_lon = np.degrees(np.arctan2(y, x))
    
    return float(actual_lon), float(actual_lat)

import numpy as np

def inverse_transform_point_corrected(visual_lon, visual_lat, center_lon, center_lat, roll=0):
    """
    Correct inverse transformation using proper spherical rotation mathematics.
    
    Based on D3's rotation convention: [lambda, phi, gamma] (lon, lat, roll)
    where rotations are applied in the order: rotate lon, then lat, then roll
    """
    # Convert to radians
    v_lon = np.radians(visual_lon)
    v_lat = np.radians(visual_lat)
    c_lon = np.radians(center_lon)
    c_lat = np.radians(center_lat)
    roll_rad = np.radians(roll)
    
    # Convert visual position to Cartesian coordinates on unit sphere
    x = np.cos(v_lat) * np.cos(v_lon)
    y = np.cos(v_lat) * np.sin(v_lon)
    z = np.sin(v_lat)
    
    # Apply inverse transformations in reverse order
    
    # 1. Inverse roll rotation (around viewing axis/z-axis in projected space)
    if roll != 0:
        cos_r = np.cos(-roll_rad)  # negative for inverse
        sin_r = np.sin(-roll_rad)
        x_new = x * cos_r - y * sin_r
        y_new = x * sin_r + y * cos_r
        x, y = x_new, y_new
    
    # 2. Inverse latitude rotation (around y-axis)
    if center_lat != 0:
        cos_lat = np.cos(-c_lat)  # negative for inverse
        sin_lat = np.sin(-c_lat)
        x_new = x * cos_lat - z * sin_lat
        z_new = x * sin_lat + z * cos_lat
        x, z = x_new, z_new
    
    # 3. Inverse longitude rotation (around z-axis)
    if center_lon != 0:
        cos_lon = np.cos(-c_lon)  # negative for inverse
        sin_lon = np.sin(-c_lon)
        x_new = x * cos_lon - y * sin_lon
        y_new = x * sin_lon + y * cos_lon
        x, y = x_new, y_new
    
    # Convert back to spherical coordinates
    actual_lat = np.degrees(np.arcsin(np.clip(z, -1, 1)))
    actual_lon = np.degrees(np.arctan2(y, x))
    
    return float(actual_lon), float(actual_lat)

import numpy as np

def inverse_transform_point_d3_style(visual_lon, visual_lat, center_lon, center_lat, roll=0):
    """
    Try to mimic D3's geoRotation inverse more closely
    """
    # D3 applies rotations as: rotate(lambda, phi, gamma)
    # where lambda=lon, phi=lat, gamma=roll
    
    # Convert to radians
    lambda_r = np.radians(-center_lon)  # Note: D3 uses negative for inverse
    phi_r = np.radians(-center_lat)
    gamma_r = np.radians(-roll)
    
    # Input point in radians
    lon_r = np.radians(visual_lon)
    lat_r = np.radians(visual_lat)
    
    # Convert to Cartesian
    x = np.cos(lat_r) * np.cos(lon_r)
    y = np.cos(lat_r) * np.sin(lon_r)
    z = np.sin(lat_r)
    
    # Apply inverse rotations in D3 order (gamma, phi, lambda)
    # Roll rotation (around z-axis)
    if roll != 0:
        cos_g, sin_g = np.cos(gamma_r), np.sin(gamma_r)
        x_new = cos_g * x - sin_g * y
        y_new = sin_g * x + cos_g * y
        x, y = x_new, y_new
    
    # Latitude rotation (around y-axis) 
    if center_lat != 0:
        cos_p, sin_p = np.cos(phi_r), np.sin(phi_r)
        x_new = cos_p * x + sin_p * z
        z_new = -sin_p * x + cos_p * z
        x, z = x_new, z_new
    
    # Longitude rotation (around z-axis)
    if center_lon != 0:
        cos_l, sin_l = np.cos(lambda_r), np.sin(lambda_r)
        x_new = cos_l * x - sin_l * y
        y_new = sin_l * x + cos_l * y
        x, y = x_new, y_new
    
    # Convert back to spherical
    actual_lat = np.degrees(np.arcsin(np.clip(z, -1, 1)))
    actual_lon = np.degrees(np.arctan2(y, x))
    
    return float(actual_lon), float(actual_lat)

In [131]:
inverse_transform_point(0, 0, 50, 40, 40)

(50.0, 40.0)

In [ ]:
inverse_transform_point(0, 0, 50, 40, 40)

In [118]:
def transform(lon, lat, center_lon, center_lat, spin, tilt, roll=0):
    x = lon - spin
    y = lat - tilt
    
    return x, y

In [119]:
def spin(lon_lat, spin=0):
    x = lon_lat[0] - spin
    y = lon_lat[1]
    
    return x, y

def tilt(lon_lat, tilt=0):
    x = lon_lat[0]
    y = lon_lat[1] - tilt
    
    return x, y

In [120]:
spin((0,90), 50)

(-50, 90)

In [144]:
inverse_transform_point(0, 89, 50, 40, 0)

(50.0, 129.0)

In [187]:
roll = 40

fig2 = plot_map(50, 40, spin=50, tilt=40, roll=roll)

n_pole = inverse_transform_point_d3_style(0, 89.99, 50, 40, roll)  # North Pole 
fig2.add_trace(go.Scattergeo(  
    lon=[n_pole[0]],
    lat=[n_pole[1]],
    mode='markers',
    marker=dict(size=10, color='red'),
    name=f'North ({round(n_pole[0])}, {round(n_pole[1])})'
))

s_pole = inverse_transform_point_d3_style(0, -89.99, 50, 40, roll)  # South Pole 
fig2.add_trace(go.Scattergeo(  
    lon=[s_pole[0]],
    lat=[s_pole[1]],
    mode='markers',
    marker=dict(size=10, color='blue'),
    name=f'South ({round(s_pole[0])}, {round(s_pole[1])})'
))

# p = inverse_transform_point(0, 0, 50, 40, roll)
# fig2.add_trace(go.Scattergeo( 
#     lon=[p[0]],
#     lat=[p[1]],
#     mode='markers',
#     marker=dict(size=10, color='green'),
#     name='_'
# ))

fig2.show()

In [99]:
transform(0, 90, center_lon=50, center_lat=40, spin=50, tilt=40, roll=0)  # > (230, 50) or (-130, 50)

(-50, 50)

In [101]:
transform(0, -90, center_lon=50, center_lat=40, spin=50, tilt=40, roll=0)  # > (50, -50)

(-50, -130)

In [159]:
np.round(89.7008, 0)

np.float64(90.0)

In [164]:
round(8.9)

9

In [182]:
# Test 1: Simple case with no rotation - this should work
fig_test1 = plot_map(0, 0, 0, 0, 0)  # no rotation at all
test_point1 = inverse_transform_point_d3_style(0, 90, 0, 0, 0)  # should be (0, 90)
print(f"No rotation test: {test_point1} (should be close to (0, 90))")

# Test 2: Only longitude rotation
fig_test2 = plot_map(30, 0, 0, 0, 0)  # only lon rotation
test_point2 = inverse_transform_point_d3_style(0, 90, 30, 0, 0)  # should be (-30, 90)
print(f"Lon rotation only: {test_point2} (should be close to (-30, 90))")

# Test 3: Only latitude rotation  
fig_test3 = plot_map(0, 30, 0, 0, 0)  # only lat rotation
test_point3 = inverse_transform_point_d3_style(0, 90, 0, 30, 0)  # what should this be?
print(f"Lat rotation only: {test_point3}")

No rotation test: (0.0, 90.0) (should be close to (0, 90))
Lon rotation only: (-29.999999999999996, 90.0) (should be close to (-30, 90))
Lat rotation only: (180.0, 60.00000000000001)
